# Toy Radix Lagrangean decomposition validation. Bisection feasibility approach (usually faster than minerr)

In [17]:
OBJECTIVE    = 'sparse'
MAX_NONZERO  = None #48*2
PREVENT_ZERO = True

In [18]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [19]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from cobra.io import load_json_model
from six import iteritems
import numpy as np
import cobra

In [20]:
ijomc = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')

In [21]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [22]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if ijomc.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

#N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS==1:
    df_conds = df_conds[ df_conds.cond=='acetate']
elif N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

# Make radix problem

In [23]:
from dynamicme.estimate import RadixEstimator

In [24]:
df_Y = df_meas.rename(columns={'growth_rate_1_h':'output', 'substrate':'cond'})
df_Y.loc[:,'output_id'] = 'BIOMASS_Ecoli_core_w_GAM'
conds = df_conds.cond.unique()
df_Y = df_Y[ df_Y.cond.isin(conds)]
df_X = df_conds

est = RadixEstimator()
est.fit(ijomc, df_X, df_Y, reg_weight = REG_WEIGHT, max_nonzero_binaries=MAX_NONZERO, optimize=False)

est.milp.Params.ScaleFlag = 0
est.milp.Params.OutputFlag = 1
est.milp.Params.TimeLimit = 2*3600.
# est.milp.Params.Presolve = 0
# est.milp.Params.FeasibilityTol = 1e-8
# est.milp.Params.OptimalityTol = 1e-8

# est.optimize()

## Validate the fit

base_model = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')

mu_id = 'BIOMASS_Ecoli_core_w_GAM'
# df_pred = est.predict(df_X, base_model, meas_id=mu_id)

df_Y2 = df_Y[[c for c in df_Y.columns if c not in ['order','ex_rxn']]]
df_val = pd.merge(df_Y2.rename(columns={'output':'meas'}), df_pred.rename(columns={'rxn':'output_id','x':'pred'}), on=['cond','output_id'])
df_val.loc[:,'error'] = (df_val.pred - df_val.meas)
df_val.loc[:,'perror'] = df_val.error / df_val.meas * 100
df_val

a0 = 1./65/3600
kfit_dict0 = {k:a0 for k in est.kfit_dict.keys()}
df_ref = est.predict(df_X, base_model, meas_id=mu_id, kfit_dict=kfit_dict0)
df_ref.rename(columns={'rxn':'output_id','x':'output'}, inplace=True)

df_val0 = pd.merge(df_Y2.rename(columns={'output':'meas'}), df_ref.rename(columns={'output':'pred0'}), on=['cond','output_id'])
df_val0.loc[:,'error0'] = (df_val0.pred0 - df_val0.meas)
df_val0.loc[:,'perror0'] = df_val0.error0 / df_val0.meas * 100

df_comp = pd.merge(df_val, df_val0, on=['cond','output_id','meas'])
df_comp.loc[:,'error_change'] = (df_comp.error.abs()-df_comp.error0.abs()) / df_comp.error0.abs() * 100
df_comp

print('Total abs error: %g' % df_comp.error.abs().sum())
print('Total perc error: %g%%' % df_comp.perror.abs().sum())
print('Total error change: %g%%' % df_comp.error_change.sum())
changed = [abs(est.kfit_dict[k]-kfit_dict0[k])>1e-9 for k in est.kfit_dict.keys()]
print('Number of keffs changed: %d/%d (%g%%)' % (sum(changed),len(est.kfit_dict), 100*sum(changed)/len(est.kfit_dict) ))

# Each Lagrange submodel should start with the optimum

In [25]:
from dynamicme.decomposition import LagrangeMaster, LagrangeSubmodel

In [26]:
conds = df_conds.cond.unique()
sub_dict = {}
for cond in conds:
    df_Xk = df_X[ df_X.cond==cond]
    df_Yk = df_Y[ df_Y.cond==cond]
    estk = RadixEstimator()
    #estk.fit(ijomc, df_Xk, df_Yk, objective='sparse', max_nonzero_binaries=MAX_NONZERO, optimize=False)    
    estk.fit(ijomc, df_Xk, df_Yk, objective='minerr', max_nonzero_binaries=MAX_NONZERO, optimize=False)    
    sub = LagrangeSubmodel(estk.stacker.model, cond)
    sub_dict[cond] = sub

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1


In [27]:
master = LagrangeMaster(estk.stacker.model)
master.add_submodels(sub_dict)

### Two-phase

In [28]:
master.time_limit = 2*3600
#master.max_iter = 500
master.print_iter = 5
master.verbosity = 1
master.solve_relaxed()
sol_best = master.optimize()

    Iter                    UB                    LB       gap relgap(%)     delta                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Dual       Feas        Sub       Best                                   total    master       sub
       0       1000       1000          0          0      1000       100       0.5  0.061236  0.001167  0.036412
       5      1.133      1.133     -46.69          0     1.133       100    0.0156  0.358707  0.001770  0.030688
      10     0.2135     0.2135     -110.7          0    0.2135       100  0.000488  0.653622  0.002320  0.026695
      15     0.1027     0.1027     -352.1          0    0.1027       100  1.53e-05  0.957796  0.003052  0.031753
      20    0.05838    0.05838       -998          0   0.05838       100  4.77e-07  1.257722  0.004377  0.026087
      25    0.04611    0.04611      -2913          0   0.04611       100  1.49e-08  1.553209  0.

/home/laurence/python_modules/dynamicme.git/dynamicme/decomposition.py:2097 UserWarning: Solution available but Master solver status=suboptimal (13).


       0 -3.954e-05 -3.954e-05   -0.02516   -0.02516   0.02512 6.353e+04       0.5  2.204864  0.267367  1.893465
       4 -3.353e-05 -3.954e-05  -0.002886 -1.822e-05-1.531e-05    -45.66    0.0625  15.08424  0.158741  1.914041
relgap (-0.456621) <= gaptol (0.0001). Finished.


## Bisect feasible error

In [12]:
master.time_limit = 2*3600
#master.max_iter = 500
master.print_iter = 5
master.verbosity = 1

max_iter = 1000
A = 0.
B = 1.

precision = 0.01
sol_best = {}

for _iter in range(max_iter):
    mid = (A+B)/2
    f_err = mid
    for i,row in df_Y.iterrows():
        sub_ind = row['cond']
        sub = master.sub_dict[sub_ind]
        mu_meas = row['output']
        out_id  = row['output_id']        
        v_out   = sub.model.getVarByName(out_id+'_%s'%sub._id)
        v_out.LB= mu_meas*(1-f_err)
        v_out.UB= mu_meas*(1+f_err)
    _ = master.solve_relaxed()
    sol_master = master.optimize()
    if sol_master is None:
        A = mid
    else:
        if B-A <= precision:
            print("Found solution within precision = %g" % precision)
            break
        else:
            B = mid
            sol_best = sol_master.copy()            
            print('*'*40)
            print('A=%g. B=%g. mid=%g.'%(A,B,mid))            

    Iter                    UB                    LB       gap relgap(%)     delta                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Dual       Feas        Sub       Best                                   total    master       sub
       0       1000       1000          0          0      1000       100       0.5  0.055690  0.001400  0.031862
       1 -1.603e-11 -1.603e-11          0          0-1.603e-11    -13.82       0.5  0.092551  0.001322  0.015132
relgap (-0.138153) <= gaptol (0.0001). Finished.
    Iter                    UB                    LB       gap relgap(%)     delta                       Time(s)
  ------   -------------------   -------------------  -------- ---------  -------- -----------------------------
               Dual       Feas        Sub       Best                                   total    master       sub
       0 -2.058e-11 -2.058e-11          4      

In [29]:
var_cons_dict = estk.var_cons_dict
powers = estk.powers
digits = estk.digits
radix  = estk.radix

In [30]:
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_best[yid]
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:    
    ijofit = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
    crowding = ijofit.metabolites.get_by_id('crowding')
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit}, combine=False)
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    err0= 100*(mu_unfiti-mu_measi)/mu_measi
    err = 100*(mu_fiti - mu_measi)/mu_measi
    derr= 100*(abs(err)-abs(err0))/abs(err0)
    print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))

Changed keffs: 38/48
Cond=glucose. mu_meas=0.74. mu_sim=0.873922 (unfit=0.873922, error=18.1%). Error=18.1% (-1.13e-10% change)
	EX_glc__D_e uptake=-10
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=0
Cond=acetate. mu_meas=0.256. mu_sim=0.255998 (unfit=0.389313, error=52.1%). Error=-0.000844% (-100% change)
	EX_glc__D_e uptake=0
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=-20


In [33]:
y_sames = []
for y in sub._ys:
    sub1 = master.sub_dict['acetate']
    sub2 = master.sub_dict['glucose']
    y1 = sub1.x_dict[y.VarName]
    y2 = sub2.x_dict[y.VarName]
    y_sames.append(abs(y1-y2)<1e-10)
    print('%12.10s%12.10s%12.10s%12.10s'%(y.VarName,y1,y2, abs(y1-y2)<1e-10))    

  binary_G6P         1.0         1.0        True
  binary_G6P         1.0         1.0        True
  binary_G6P         0.0         0.0        True
  binary_AKG         1.0         1.0        True
  binary_AKG         1.0         1.0        True
  binary_AKG         1.0         1.0        True
  binary_ACK         1.0         1.0        True
  binary_ACK         1.0         1.0        True
  binary_ACK         1.0         1.0        True
  binary_MAL         0.0         0.0        True
  binary_MAL         0.0         0.0        True
  binary_MAL         1.0         1.0        True
  binary_GLU         1.0         1.0        True
  binary_GLU         0.0         0.0        True
  binary_GLU         0.0         0.0        True
  binary_ME1         0.0         0.0        True
  binary_ME1         0.0         0.0        True
  binary_ME1         1.0         1.0        True
  binary_GND         0.0         0.0        True
  binary_GND         1.0         1.0        True
  binary_GND        

In [34]:
print('num same = %s/%s'%(sum(y_sames), len(y_sames)))

num same = 144/144
